In [9]:
%load_ext autoreload
%autoreload 2
import torch
from augment import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
space = AugmentationSpace()

In [11]:
test_data = torch.rand((10, 3, 32, 32))

In [12]:
policy = [
    [  # Sub-policy 1
        {'operation': 5, 'magnitude': 3, 'probability': 0.7},  # Cutout at 70%
        {'operation': 3, 'magnitude': 2, 'probability': 0.8}   # Shear_x at 80%
    ],
    [  # Sub-policy 2
        {'operation': 1, 'magnitude': 4, 'probability': 0.3},  # Translate_x at 30%
        {'operation': 3, 'magnitude': 1, 'probability': 0.9}   # Brightness at 90%
    ]
]

In [13]:
augmented_data = apply_auto_augmentations(test_data, policy, space)
augmented_data.shape

torch.Size([10, 3, 32, 32])

In [14]:
augmented_data = apply_random_augmentations(test_data, space)
augmented_data.shape

torch.Size([10, 3, 32, 32])

## Results of AutoAugment Training a Resnet-18 with a PPO Agent
### Objective
Three models were trained in parallel (RL-augmented, randomly augmented, no augmentation) to classify images from the CIFAR-10 dataset, choosing one out of ten classes. The goal was to (1) reproduce results using an similar approach and (2) add a baseline comparison - a model trained with random augmentation - to assess whether RL-augmented training truly strengthens generalization.
### Description of DAC using RL
Dynamic algorithm configuration in essence means optimizing any algorithm (mostly DL) dynamically. Usually, that means optimize specific parameters on-the-fly during training. In this case, tuning the strategy of augmenting images that are fed into a CNN. This could involve cropping, flipping, mirroring or changing contrast. The RL agent should learn a policy during training, which augmentation strategies (the actions) out of a search space (which is the same as the baseline) are optimally so that the learner (the CNN) generalizes well on the validation set. This can be seen as a grey-box optimization problem, because the agent can see the training dynamics (instead of only seeing the result AFTER the learner was trained). Whether or not the computational expense are worth the result can only show in validating with the held-out set.

## Get files

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from main import METRICS_PATH, SEEDS

trainingmetrics_list = []
testmetrics_list = []
agentmetrics = []

for seed in SEEDS:
    METRICS_FILENAME = f"final_metrics_seednr_{seed}.csv"
    TESTMETRICS_FILENAME = f"final_testmetrics_seednr_{seed}.csv"
    AGENTPOLICIES_FILENAME = f"agent_policies_seednr_{seed}.csv"
    
    try:
        # Create paths
        trainingmetrics_path = os.path.join(METRICS_PATH, METRICS_FILENAME)
        testmetrics_path = os.path.join(METRICS_PATH, TESTMETRICS_FILENAME)
        agentmetrics_path = os.path.join(METRICS_PATH, AGENTPOLICIES_FILENAME)
        
        # Get files
        trainingmetrics_list.append(pd.read_csv(trainingmetrics_path))
        testmetrics_list.append(pd.read_csv(testmetrics_path))
        agentmetrics.append(pd.read_csv(agentmetrics_path))
    except Exception as e:
        print(f"Files do not exist: {e}")

Files do not exist: [Errno 2] No such file or directory: 'src/data/metrics/final_metrics_seednr_42.csv'
Files do not exist: [Errno 2] No such file or directory: 'src/data/metrics/final_metrics_seednr_123.csv'
Files do not exist: [Errno 2] No such file or directory: 'src/data/metrics/final_metrics_seednr_456.csv'


## Plots

In [ ]:
# Combine all training metrics and add seed info
for i, df in enumerate(trainingmetrics_list):
    df['seed'] = SEEDS[i]
combined_training = pd.concat(trainingmetrics_list, ignore_index=True)

# Same for test metrics
for i, df in enumerate(testmetrics_list):
    df['seed'] = SEEDS[i]
combined_test = pd.concat(testmetrics_list, ignore_index=True)

# Create plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Training accuracy comparison across methods
sns.boxplot(data=combined_test, x='method', y='accuracy', ax=axes[0,0])
axes[0,0].set_title('Final Test Accuracy by Method')
axes[0,0].set_ylabel('Accuracy')

# Plot 2: Training loss over time
if 'epoch' in combined_training.columns:
    for method in ['rl', 'random', 'none']:
        method_data = combined_training[combined_training['method'] == method]
        axes[0,1].plot(method_data.groupby('epoch')['loss'].mean(), label=method)
    axes[0,1].set_title('Training Loss Over Time')
    axes[0,1].legend()

# Plot 3: Statistical comparison
method_stats = combined_test.groupby('method')['accuracy'].agg(['mean', 'std'])
axes[1,0].bar(method_stats.index, method_stats['mean'], yerr=method_stats['std'])
axes[1,0].set_title('Mean Accuracy ± Std')
axes[1,0].set_ylabel('Accuracy')

# Plot 4: Agent reward trajectory
if agentmetrics:
    combined_agent = pd.concat(agentmetrics, ignore_index=True)
    axes[1,1].plot(combined_agent.groupby('update')['reward'].mean())
    axes[1,1].set_title('Agent Reward Over Updates')

plt.tight_layout()
plt.show()